In [2]:
import pandas as pd
from Bio import SeqIO

from common.env_config import config

In [3]:
import os

train_df = pd.read_csv(os.path.join(config.DATA_DIR, 'custom/train.csv'))
test_df = pd.read_csv(os.path.join(config.DATA_DIR, 'custom/test.csv'))
train_df.head()

,accession_number,lifecycle_deephage,note_deephage,source_deephage,lifecycle_deep_pl,note_deep_pl,source_deep_pl,_merge,is_valid,label
0,NC_041998,Lytic,dataset_2,deephage,Lytic,Training,deep_pl,both,valid,Lytic
1,NC_004462,Lysogenic,dataset_2,deephage,Lysogenic,Training,deep_pl,both,valid,Lysogenic
2,NC_020862,Lytic,dataset_2,deephage,Lytic,Training,deep_pl,both,valid,Lytic
3,NC_041853,Lysogenic,dataset_2,deephage,Lysogenic,Training,deep_pl,both,valid,Lysogenic
4,NC_019443,Lytic,dataset_2,deephage,Lytic,Training,deep_pl,both,valid,Lytic


In [4]:
train_df['accession_number'].duplicated().sum()

2

In [5]:
# Đường dẫn đến thư mục chứa file GenBank
gb_dir = config.GEN_BANK_DIR


# Hàm đọc file GenBank (đã có trong code của bạn)
def read_genbank_file(row):
    filename = f'{row["accession_number"]}_{row["label"]}.gb'
    try:
        record = SeqIO.read(f'{gb_dir}/{filename}', "genbank")
        record.id = f'{row["accession_number"]}|{row["label"]}'
        return record
    except FileNotFoundError:
        return None


def get_sequence(row):
    filename = f'{row["accession_number"]}_{row["label"]}.gb'
    try:
        record = SeqIO.read(f'{gb_dir}/{filename}', "genbank")
        return record.seq
    except FileNotFoundError:
        return None


def labeling(row):
    if row["label"] == "Lysogenic":
        return 0
    elif row["label"] == "Lytic":
        return 1


output_train_fasta = "../data/my_data/aggregated_deephage_deeppl_data/train.fasta"
output_test_fasta = "../data/my_data/aggregated_deephage_deeppl_data/test.fasta"

train_df["genbank_record"] = train_df.apply(read_genbank_file, axis=1)
test_df["genbank_record"] = test_df.apply(read_genbank_file, axis=1)

train_df["sequence"] = train_df.apply(lambda row: get_sequence(row), axis=1)
test_df["sequence"] = test_df.apply(lambda row: get_sequence(row), axis=1)

train_df["target"] = train_df.apply(lambda row: labeling(row), axis=1)
test_df["target"] = test_df.apply(lambda row: labeling(row), axis=1)

# train_df["sequence", "target"].to_csv(output_train_fasta, index=False)
# test_df["sequence", "target"].to_csv(output_test_fasta, index=False)

# Lọc bỏ các bản ghi None và ghi vào file FASTA
# train_records_to_write = [record for record in train_df["genbank_record"] if record is not None]
# test_records_to_write = [record for record in test_df["genbank_record"] if record is not None]
#
# # Ghi vào file FASTA
# SeqIO.write(train_records_to_write, output_train_fasta, "fasta")
# SeqIO.write(test_records_to_write, output_test_fasta, "fasta")
#
# print(f"Đã ghi {len(train_records_to_write)} trình tự vào file {output_train_fasta}")
# print(f"Đã ghi {len(test_records_to_write)} trình tự vào file {output_test_fasta}")

In [6]:
train_df.head()

,accession_number,lifecycle_deephage,note_deephage,source_deephage,lifecycle_deep_pl,note_deep_pl,source_deep_pl,_merge,is_valid,label,genbank_record,sequence,target
0,NC_041998,Lytic,dataset_2,deephage,Lytic,Training,deep_pl,both,valid,Lytic,"(C, C, T, T, C, A, C, G, C, A, A, C, C, A, C, ...","(C, C, T, T, C, A, C, G, C, A, A, C, C, A, C, ...",1
1,NC_004462,Lysogenic,dataset_2,deephage,Lysogenic,Training,deep_pl,both,valid,Lysogenic,"(G, G, A, T, C, C, C, T, C, A, A, C, A, G, G, ...","(G, G, A, T, C, C, C, T, C, A, A, C, A, G, G, ...",0
2,NC_020862,Lytic,dataset_2,deephage,Lytic,Training,deep_pl,both,valid,Lytic,"(A, C, G, A, A, T, A, T, T, C, T, T, T, G, C, ...","(A, C, G, A, A, T, A, T, T, C, T, T, T, G, C, ...",1
3,NC_041853,Lysogenic,dataset_2,deephage,Lysogenic,Training,deep_pl,both,valid,Lysogenic,"(T, G, C, G, G, C, T, G, C, C, A, G, A, T, C, ...","(T, G, C, G, G, C, T, G, C, C, A, G, A, T, C, ...",0
4,NC_019443,Lytic,dataset_2,deephage,Lytic,Training,deep_pl,both,valid,Lytic,"(A, T, G, T, C, T, T, T, C, G, C, A, T, C, A, ...","(A, T, G, T, C, T, T, T, C, G, C, A, T, C, A, ...",1


In [8]:
train_df[["sequence", "target"]].dropna().to_csv(os.path.join(config.DATA_DIR, "custom/train_ds.csv"), index=False)
test_df[["sequence", "target"]].dropna().to_csv(os.path.join(config.DATA_DIR, "custom/test_ds.csv"), index=False)

In [16]:
# Đọc và hiển thị thông tin từ file FASTA
fasta_records = list(SeqIO.parse(output_train_fasta, "fasta"))

print(f"Đã đọc {len(fasta_records)} trình tự từ file {output_train_fasta}")

# Hiển thị thông tin về các trình tự
for i, record in enumerate(fasta_records):
    print(f"Record {i + 1}:")
    print(f"  ID: {record.id}")
    print(f"  Description: {record.description}")
    print(f"  Độ dài trình tự: {len(record.seq)} bp")
    # Hiển thị 50 ký tự đầu tiên của trình tự
    print(f"  Trình tự (50 ký tự đầu): {record.seq[:50]}...")
    print()

# Để kiểm tra tính toàn vẹn của dữ liệu
print("Thống kê độ dài trình tự:")
lengths = [len(record.seq) for record in fasta_records]
print(f"  Trình tự ngắn nhất: {min(lengths)} bp")
print(f"  Trình tự dài nhất: {max(lengths)} bp")
print(f"  Độ dài trung bình: {sum(lengths) / len(lengths):.2f} bp")

Đã đọc 1733 trình tự từ file ../data/my_data/aggregated_deephage_deeppl_data/train.fasta
Record 1:
  ID: NC_020477|Lytic
  Description: NC_020477|Lytic Bacillus phage Eoghan, complete genome
  Độ dài trình tự: 49458 bp
  Trình tự (50 ký tự đầu): CCCCCACCCCGATGAGGTCGCTTCCCTTGTTCTGGAGACCTGTTAGAAAA...

Record 2:
  ID: NC_042325|Lysogenic
  Description: NC_042325|Lysogenic Mycobacterium virus Pixie, complete genome
  Độ dài trình tự: 61147 bp
  Trình tự (50 ký tự đầu): AGGCGGGTTTCTCTCCCCGGCATTTTTTCCCAAACCGAAAGGTCAGCGCA...

Record 3:
  ID: NC_041955|Lytic
  Description: NC_041955|Lytic Propionibacterium phage PHL082M03, complete genome
  Độ dài trình tự: 29491 bp
  Trình tự (50 ký tự đầu): AGTGAAATACCTCCCTTTTTTGGTTTTGTCTGTTTGTCGACTTTTTGTGT...

Record 4:
  ID: NC_021862|Lytic
  Description: NC_021862|Lytic Ralstonia phage RSS30 DNA, complete genome
  Độ dài trình tự: 8576 bp
  Trình tự (50 ký tự đầu): ATGCAGCAGAGAGACCATAGGCAGCTGGACCTGATTGGCGACGTGCTTCA...

Record 5:
  ID: NC_001421|Lytic
  De